# ML model Training (Question 3 of the Homework)

In [1]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Read datasets

df_pe = pd.read_csv("../data/pe/cleaned/pe_dataset.csv")
normalized_df_pe = pd.read_csv("../data/pe/cleaned/normalized_pe_dataset.csv")
normalized_df_pe_without_outliers = pd.read_csv("../data/pe/cleaned/normalized_pe_dataset_without_outliers.csv")

## Linear Regression

As seen in the [ML algorithm selection notebook](ML_algorithm_selection.ipynb), the linear model does not have any hyperparameters to tune. Therefore, cross-validation is not required. We can split our datasets into training, testing and validation sets like so:

- Training set: 70%
- Testing set: 15%
- Validation set: 15%

To do that, I'll use the `train_test_split` function from `sklearn.model_selection` module. This function provides parameters to shuffle or not our dataset before splitting it. I'll set the `shuffle` parameter to `True` to ensure that the data is shuffled before splitting it. According to [scikit-learn documentation](https://scikit-learn.org/stable/glossary.html#term-random_state), by default the function will use the global random state from `np.random` module to shuffle the data. 

NOTE: I don't see the use here of having reproducible results, so I'll leave the `random_state` parameter to `None`.

In [11]:
def splitDataset(df, test_size=0.15):
    from sklearn.model_selection import train_test_split
    X = df.drop(columns=['Entity', 'Year','Primary energy consumption per capita (kWh/person)'])
    y = df['Primary energy consumption per capita (kWh/person)']
    X_pretrain, X_test, y_pretrain, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=None)
    X_train, X_val, y_train, y_val = train_test_split(X_pretrain, y_pretrain, test_size=test_size, shuffle=True, random_state=None)
    print(X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape)
    return X_train, X_test, X_val, y_train, y_test, y_val

In [13]:
# Split datasets into train, test and validation sets
X_pe_train, X_pe_test, X_pe_val, y_pe_train, y_pe_test, y_pe_val = splitDataset(df_pe)

# nor -> normalized
X_nor_pe_train, X_nor_pe_test, X_nor_pe_val, y_nor_pe_train, y_nor_pe_test, y_nor_pe_val = splitDataset(normalized_df_pe)

# nor_wo -> normalized without outliers
X_nor_pe_wo_train, X_nor_pe_wo_test, X_nor_pe_wo_val, y_nor_pe_wo_train, y_nor_pe_wo_test, y_nor_pe_wo_val = splitDataset(normalized_df_pe_without_outliers)


(2306, 4) (479, 4) (408, 4) (2306,) (479,) (408,)
(2306, 4) (479, 4) (408, 4) (2306,) (479,) (408,)
(2173, 4) (452, 4) (384, 4) (2173,) (452,) (384,)
